# Components

Components:
 - Convert file to Parquet (.to_parquet)
 - Export data from BQ as parquet to GCS
 - Fit the data (.fit)
 - Preprocess the data (.transform)

All the components will use FUSE as the main storage driver.  
Files will be read and write from/to GCS after each operation.

Components must be generic enough for customers to reuse them.